### Geography Cleaning Steps

- The goal of this notebook is to get organized with the geography cleaning steps to then port over into the repo

### Set Up

In [1]:
pip install geocoder


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

pd.set_option('display.max_rows', 500)

### Load CDOT Data

In [6]:
import sys  
sys.path.insert(1, '../')

from convert_old_cdot_format_to_new_format import convert_old_cdot_format_to_new_format

In [7]:
# Load CDOT Data for a list of years

years = [2020, 2021]

cdot_base_path = "/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/"

loaded_data_list = []
for y in years:
    filename = f"CDOTRM_CD_Crash_Listing_-_{y}.xlsx"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_excel(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)
    
    if y < 2021:
        # call conversion code 
        year_cdot_pdf = convert_old_cdot_format_to_new_format(year_cdot_pdf)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list).reset_index()

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2020.xlsx
['SYSTEM', 'RTE', 'SEC', 'MP', 'DATE']
(86304, 85)


/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/../convert_old_cdot_format_to_new_format.py:403: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = ''
/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/../convert_old_cdot_format_to_new_format.py:403: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = ''
/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/../convert_old_cdot_format_to_new_format.py:403: Perform

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2021.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(97848, 111)


### Functions

In [8]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

def get_city_from_coordinates(latitude, longitude):
    """
    Retrieves the city from given latitude and longitude.
    """
    geolocator = Nominatim(user_agent="hmn") # Replace "my_geocoding_app" with a unique identifier
    try:
        location = geolocator.reverse((latitude, longitude), exactly_one=True)
        if location:
            address = location.raw.get('address', {})
            city = address.get('city') or \
                   address.get('town') or \
                   address.get('village')
            return city
        else:
            return "none"
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Geocoding error: {e}")
        return "none"

### Geography Cleaning

In [9]:
print('Loaded Data Size', cdot_pdf_load.shape)

Loaded Data Size (184152, 150)


In [10]:
# STEP 1 - Drop duplicates
cdot_pdf_withNans_dedup = cdot_pdf_load.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)

display(cdot_pdf_withNans_dedup.head(5))

Data size after drop duplicates (184152, 150)


,index,System Code,Crash Date,Injury 00,Injury 01,Injury 02,Injury 03,Injury 04,Agency Id,Location,...,Wild Animal,Road Contour Curves,Road Contour Grade,Number Killed,Number Injured,Approach Overtaking Turn,Crash Time,Rd_Number,Rd_Section,City_Street
0,0,County Road,2020-01-01,0.0,0.0,0.0,0.0,0.0,CSP,On Roadway,...,NaN,Straight,Level,0.0,0.0,Not Applicable,00:00:00,NaN,SYCAM,NaN
1,1,City Street,2020-01-01,0.0,0.0,0.0,0.0,0.0,EGLSO,Ran off right side,...,NaN,Curve Unknown Direction,Level,0.0,0.0,Not Applicable,10:00:00,NaN,NIGHT,NaN
2,2,State Highway,2020-01-01,1.0,1.0,0.0,0.0,0.0,CSP,On Roadway,...,ELK,Straight,Level,0.0,1.0,Not Applicable,10:00:00,160.0A,123.1,NaN
3,3,City Street,2020-01-01,0.0,0.0,0.0,0.0,0.0,ASO,On Roadway,...,NaN,Straight,Level,0.0,0.0,Not Applicable,60:00:00,NaN,JAMIS,NaN
4,4,City Street,2020-01-01,0.0,1.0,0.0,0.0,0.0,DPD,Ran off right side,...,NaN,Straight,Level,0.0,1.0,Not Applicable,13:00:00,NaN,AURAR,NaN


In [11]:
# STEP 2 - fill NaN and null values in geography columns 
cdot_pdf = cdot_pdf_withNans_dedup.copy()

# check NaN count by column 
for col in ['City', 'County', 'Crash Year', 'Crash Date', 'Latitude', 'Longitude']:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
print()

# Fill NaN values 
string_geography_cols = ['Crash Date', 'Crash Year', 'City', 'County']
cdot_pdf[string_geography_cols] = cdot_pdf[string_geography_cols].fillna('none')
cdot_pdf['Latitude'] = cdot_pdf['Latitude'].fillna(999.999000)
cdot_pdf['Longitude'] = cdot_pdf['Longitude'].fillna(999.999000)

# check NaN count by column
print('after handle nans')
for col in ['City', 'County', 'Crash Year', 'Crash Date', 'Latitude', 'Longitude']:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])

City 45445
County 97
Crash Year 97
Crash Date 97
Latitude 23932
Longitude 23932

after handle nans
City 0
City 0
County 0
County 0
Crash Year 0
Crash Year 0
Crash Date 0
Crash Date 0
Latitude 0
Latitude 0
Longitude 0
Longitude 0


In [14]:
# STEP 3 - handle weird corner cases 

# mark "unknown" city as 'none'
cdot_pdf.loc[cdot_pdf['City'].str.contains('UNKNOWN'), 'City'] = 'none'

# set default Denver latitude and longitude value to 999.999000
default_lat = 39.74602
default_long = -104.98877
default_lat_long_condition = (cdot_pdf['Latitude'] == default_lat) & (cdot_pdf['Longitude'] == default_long)
cdot_pdf.loc[default_lat_long_condition, 'Latitude'] = 999.999000
cdot_pdf.loc[default_lat_long_condition, 'Longitude'] = 999.999000

# set 000 Latitude and Longitude to 999.999000
zero_lat_long_condition = (cdot_pdf['Latitude'].round() == 0) & (cdot_pdf['Longitude'].round() == 0)
cdot_pdf.loc[zero_lat_long_condition, 'Latitude'] = 999.999000
cdot_pdf.loc[zero_lat_long_condition, 'Longitude'] = 999.999000

# confirm no corner cases remain
print(cdot_pdf.loc[cdot_pdf['City'].str.contains('UNKNOWN')].shape)
print(cdot_pdf.loc[(cdot_pdf['Latitude'] == default_lat) & (cdot_pdf['Longitude'] == default_long)].shape)
print(cdot_pdf.loc[(cdot_pdf['Latitude'].round() == 0) & (cdot_pdf['Longitude'].round() == 0)].shape)

(0, 150)
(0, 150)
(0, 150)


In [15]:
# STEP 4 - Fill missing city values 

# filter to El Paso County for time saving **
cdot_pdf_elpaso = cdot_pdf.loc[cdot_pdf['County'] == 'EL PASO']

# remove missing Latitude and Longitude info
cdot_pdf_w_lat_long = cdot_pdf_elpaso.loc[~(cdot_pdf_elpaso['Latitude'] == 999.999000) & ~(cdot_pdf_elpaso['Longitude'] == 999.999000)]

# filter to where city is not provided and see how often we can get the associated city
cdot_pdf_w_latlong_and_no_city = cdot_pdf_w_lat_long.loc[cdot_pdf_w_lat_long['City'] == 'none']
display(cdot_pdf_w_latlong_and_no_city[['Latitude', 'Longitude', 'City']].head(10))

# get city from lat long
cdot_pdf_w_latlong_and_no_city['City'] = cdot_pdf_w_latlong_and_no_city.apply(
        lambda row: get_city_from_coordinates(row['Latitude'], row['Longitude']),
        axis=1,
    )

display(cdot_pdf_w_latlong_and_no_city[['Latitude', 'Longitude', 'City']].head(50))

,Latitude,Longitude,City
18,38.715241,-104.726856,none
178,38.748394,-104.819872,none
372,39.120996,-104.865189,none
373,39.119588,-104.865425,none
374,39.119588,-104.865425,none


,Latitude,Longitude,City
18,38.715241,-104.726856,Fountain
178,38.748394,-104.819872,Colorado Springs
372,39.120996,-104.865189,Woodmoor
373,39.119588,-104.865425,Woodmoor
374,39.119588,-104.865425,Woodmoor


In [26]:
# Join back in the new city values 
cdot_pdf.loc[cdot_pdf_w_latlong_and_no_city.index, 'City'] = cdot_pdf_w_latlong_and_no_city['City'].str.upper().values

print(cdot_pdf_w_latlong_and_no_city['City'].values)
print(cdot_pdf_w_latlong_and_no_city.index)
print(cdot_pdf.loc[cdot_pdf_w_latlong_and_no_city.index].City)

['Fountain' 'Colorado Springs' 'Woodmoor' 'Woodmoor' 'Woodmoor']
Index([18, 178, 372, 373, 374], dtype='int64')
18             FOUNTAIN
178    COLORADO SPRINGS
372            WOODMOOR
373            WOODMOOR
374            WOODMOOR
Name: City, dtype: object


In [23]:
# STEP 5 - Create truncated latitude and longitude for analysis 
cdot_pdf.loc[:, 'Latitude_3dec'] = cdot_pdf['Latitude'].round(3)
cdot_pdf.loc[:, 'Longitude_3dec'] = cdot_pdf['Longitude'].round(3)

display(cdot_pdf[['Latitude', 'Latitude_3dec', 'Longitude', 'Longitude_3dec']].head(3))

,Latitude,Latitude_3dec,Longitude,Longitude_3dec
0,999.999000,999.999,999.999000,999.999
1,999.999000,999.999,999.999000,999.999
2,37.204663,37.205,-107.329465,-107.329


In [39]:
# STEP 6 -  combine location 1 and location 2
# more useful to understand the intersection vs. location 1 at location 2 being different than location 2 at location 2. 
# *************************

# determine if Location 1 is first alphabetically
cdot_pdf.loc[:, 'loc1_first'] = cdot_pdf.loc[:, 'Location 1'] > cdot_pdf.loc[:, 'Location 2'] 

# combine Location 1 and Location 2 alphabetically 
# location 1 is first
cdot_pdf.loc[cdot_pdf['loc1_first'], 'alph_loc1_loc2'] = \
        cdot_pdf.loc[cdot_pdf['loc1_first'], 'Location 1'] + \
        ' ' + \
        cdot_pdf.loc[cdot_pdf['loc1_first'], 'Location 2']
# location 2 is second 
cdot_pdf.loc[~cdot_pdf['loc1_first'], 'alph_loc1_loc2'] = \
        cdot_pdf.loc[~cdot_pdf['loc1_first'], 'Location 2'] + \
        ' ' + \
        cdot_pdf.loc[~cdot_pdf['loc1_first'], 'Location 1']

# check result
display(cdot_pdf[['Location 1', 'Location 2', 'loc1_first', 'alph_loc1_loc2']].head(5))

,Location 1,Location 2,loc1_first,alph_loc1_loc2
0,SYCAMORE CT,WELLINGTON AVE,False,WELLINGTON AVE SYCAMORE CT
1,NIGHTHAWK CIR,41 NIGHTHAWK CIR,True,NIGHTHAWK CIR 41 NIGHTHAWK CIR
2,HWY 160,MM 123,False,MM 123 HWY 160
3,JAMISON DR 15700 E,JASPER CT,False,JASPER CT JAMISON DR 15700 E
4,AURARIA PK,9TH ST,True,AURARIA PK 9TH ST


In [46]:
print(cdot_pdf.loc[(cdot_pdf['Latitude'] == 999.999000) & (cdot_pdf['Longitude'] == 999.999000)].shape)

display(cdot_pdf.loc[(cdot_pdf['Latitude'] == 999.999000) & (cdot_pdf['Longitude'] == 999.999000), ['Latitude', 'Longitude', 'alph_loc1_loc2']])


(35883, 155)


,Latitude,Longitude,alph_loc1_loc2
0,999.999,999.999,WELLINGTON AVE SYCAMORE CT
1,999.999,999.999,NIGHTHAWK CIR 41 NIGHTHAWK CIR
8,999.999,999.999,CROW CUTOFF BONDURANT RD
12,999.999,999.999,LITTLE BEAVER TRL 190 LITTLE BEAVER TRL
16,999.999,999.999,ROLLER COASTER RD MOUNTAIN PINE LN
...,...,...,...
184122,999.999,999.999,S COLLYER ST CHALLENGER PL
184123,999.999,999.999,EUCLID AVE 8TH ST
184126,999.999,999.999,N CHAMBERS RD E6TH AVE
184139,999.999,999.999,S EVANSTON ST E MEXICO DR


In [45]:
display(cdot_pdf.loc[cdot_pdf['alph_loc1_loc2'] == 'WIDEFIELD BLVD FONTAINE BLVD', ['Latitude_3dec', 'Longitude_3dec', 'alph_loc1_loc2']])


,Latitude_3dec,Longitude_3dec,alph_loc1_loc2
635,999.999,999.999,WIDEFIELD BLVD FONTAINE BLVD
19459,999.999,999.999,WIDEFIELD BLVD FONTAINE BLVD
66640,999.999,999.999,WIDEFIELD BLVD FONTAINE BLVD
134822,38.737,-104.729,WIDEFIELD BLVD FONTAINE BLVD
138163,999.999,999.999,WIDEFIELD BLVD FONTAINE BLVD
138226,999.999,999.999,WIDEFIELD BLVD FONTAINE BLVD
171605,38.737,-104.729,WIDEFIELD BLVD FONTAINE BLVD


In [66]:
# Create the mapping dictionary from combined loc1/2 column to 3 decimal lat long
alph_and_latitude = cdot_pdf.loc[~(cdot_pdf['Latitude'] == 999.999000) & ~(cdot_pdf['Longitude'] == 999.999000), ['alph_loc1_loc2', 'Latitude_3dec']].drop_duplicates()
display(alph_and_latitude.head(10))

temp = alph_and_latitude.groupby('alph_loc1_loc2').count()
print(temp.shape)
al12_w_dup_lat = temp.loc[temp['Latitude_3dec'] > 1].reset_index()
display(al12_w_dup_lat)
list_duplicate_al12 = al12_w_dup_lat['alph_loc1_loc2'].values

alph_and_latitude_filt = alph_and_latitude.loc[~alph_and_latitude['alph_loc1_loc2'].isin(list_duplicate_al12)]
print(alph_and_latitude_filt.groupby('alph_loc1_loc2').count().shape)

mapping_dict = alph_and_latitude_filt.set_index(['alph_loc1_loc2'])['Latitude_3dec'].to_dict()
print(mapping_dict)

,alph_loc1_loc2,Latitude_3dec
2,MM 123 HWY 160,37.205
3,JASPER CT JAMISON DR 15700 E,39.579
4,AURARIA PK 9TH ST,39.747
5,I-25 9500 BLK,40.135
6,SANTA FE DR EXIT GORE PT I-25 SB,39.706
7,CR 4 CR 15,40.012
9,VERDE DR CHELTON RD S,38.814
10,MM 230 I-25,40.013
11,MALAYA ST N 40TH AVE 20800 BLK E,39.771
13,HAVANA ST S ALAMEDA AVE E,39.711


(83114, 1)


,alph_loc1_loc2,Latitude_3dec
0,100 BLK E 84TH AVE 100 BLK E,2
1,11TH AVE 10TH ST,2
2,12TH AVE 10TH ST,2
3,12TH ST 12TH AVE,2
4,13TH ST 11TH AVE,3
...,...,...
7660,ZUNI ST HWY 36,2
7661,ZUNI ST N 35TH AVE W,2
7662,ZUNI ST N 37TH AVE W,2
7663,ZUNI ST N SPEER BLVD N,2


(75449, 1)
{'JASPER CT JAMISON DR 15700 E': 39.579, 'SANTA FE DR EXIT GORE PT I-25 SB': 39.706, 'CR 4 CR 15': 40.012, 'MALAYA ST N 40TH AVE 20800 BLK E': 39.771, 'SANTA FE AVE S C ST E': 38.257, 'GRANDVIEW AVE AMMONS ST 5625': 39.798, 'I-70 CENTRAL PARK N': 39.777, 'ZUNI ST S VASSAR AVE 2400 BLK W': 39.67, 'RAILROAD ST BLVD B': 40.574, 'CHAMBERS RD N ANDREWS DR E': 39.775, 'SPEER BLVD N I-25 OVERPASS': 39.755, 'SAULSBURY ST S PAYNE AVE W': 39.562, 'GRANT RANCH B BOWLES AVE W': 39.61, 'CHELTON RD S BRIDGEWOOD LN': 38.809, 'PINTAIL D EAGLEVIEW C': 40.133, 'PLATTE AVE E CIRCLE DR N': 38.84, '14TH AVE 11TH ST': 40.421, 'HWY 34 BUS 43RD AVE': 40.422, 'WOODMEN RD E 2555 E WOODMEN RD': 38.933, 'YULE ST 82ND AVE W': 39.845, 'CHAMBERS RD N 17TH AVE E': 39.744, 'PEORIA ST COLFAX AVE': 39.74, 'SHERIDAN BLVD 2900 BLK S BATES AVE W': 39.659, 'JEWELL AVE E DOWNING ST S': 39.682, 'LIMA ST S JEWELL AVE E': 39.682, 'TOWER RD N PENA B 19150': 39.084, 'CR 24 7489 CR 24': 40.16, 'MM 247 HWY 160': 37.475, 

In [67]:
cdot_pdf['Latitude_3dec'] = cdot_pdf.apply(lambda row: mapping_dict.get((row['alph_loc1_loc2'])), axis=1)


In [69]:
print(cdot_pdf.loc[(cdot_pdf['Latitude_3dec'] == 999.999)].shape)

cdot_pdf['Latitude_3dec'] = cdot_pdf.apply(lambda row: mapping_dict.get((row['alph_loc1_loc2'])), axis=1)

print(cdot_pdf.loc[(cdot_pdf['Latitude_3dec'] == 999.999)].shape)

(0, 155)
(0, 155)


In [70]:
display(cdot_pdf.loc[cdot_pdf['alph_loc1_loc2'] == 'WIDEFIELD BLVD FONTAINE BLVD', ['Latitude_3dec', 'Longitude_3dec', 'alph_loc1_loc2']])


,Latitude_3dec,Longitude_3dec,alph_loc1_loc2
635,38.737,999.999,WIDEFIELD BLVD FONTAINE BLVD
19459,38.737,999.999,WIDEFIELD BLVD FONTAINE BLVD
66640,38.737,999.999,WIDEFIELD BLVD FONTAINE BLVD
134822,38.737,-104.729,WIDEFIELD BLVD FONTAINE BLVD
138163,38.737,999.999,WIDEFIELD BLVD FONTAINE BLVD
138226,38.737,999.999,WIDEFIELD BLVD FONTAINE BLVD
171605,38.737,-104.729,WIDEFIELD BLVD FONTAINE BLVD


In [73]:
display(cdot_pdf.loc[(cdot_pdf['Latitude'] == 999.999000) & ~(cdot_pdf['Latitude_3dec'].isna()), ['Latitude', 'Latitude_3dec', 'alph_loc1_loc2']])


,Latitude,Latitude_3dec,alph_loc1_loc2
84,999.999,39.466,UPPER CATTLE CREEK RD MM 1
219,999.999,37.222,MM 3 CR 141
282,999.999,39.411,UPPER CATTLE CREEK RD MM 2
340,999.999,39.833,PENA BLVD 75TH AVE E
364,999.999,38.257,LAKE AVE ABRIENDO AVE E
...,...,...,...
183982,999.999,39.799,NEVADA ST 100 BLK
184058,999.999,40.162,TERRY ST 2ND AVE
184064,999.999,39.711,W ALAMEDA AVE S PIERCE ST
184081,999.999,39.463,CR 166 CR 13
